## Importing stuff

In [1]:
import sys, os
import tsai
from tsai.all import *
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
sys.path.append(os.path.abspath('../../'))
from utils import utils
sys.path.append(os.path.abspath('../'))
from timeseries_utils import *

## Printing configs

In [4]:
print('tsai       :', tsai.__version__)
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

tsai       : 0.1.0
fastai2    : 0.0.17
fastcore   : 0.1.17
torch      : 1.4.0
scipy      : 1.4.1
numpy      : 1.18.1
pandas     : 1.0.4
Total RAM  : 62.75 GB
Used RAM   :  4.04 GB
n_cpus     : 16
device     : cuda (GeForce GTX 1080)


## Loading data

In [5]:
path_to_data='../../../'

In [6]:
data=np.load(os.path.abspath(path_to_data+'data-002.npy'),allow_pickle=True)
Y=np.load(path_to_data+'Y.npy',allow_pickle=True)
labels =pd.read_csv(path_to_data+'labels.csv')

train_fold=8
val_fold=9
test_fold=10

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == test_fold]
y_test = Y[labels.strat_fold == test_fold]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == val_fold]
y_val = Y[labels.strat_fold == val_fold]
# rest for training
X_train = data[labels.strat_fold <= train_fold]
y_train = Y[labels.strat_fold <= train_fold]

In [ ]:
# Preprocess signal data
X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test,'/content/')
n_classes = y_train.shape[1]
X_train = np.reshape(X_train,[X_train.shape[0],X_train.shape[2],X_train.shape[1]])
X_val = np.reshape(X_val,[X_val.shape[0],X_val.shape[2],X_val.shape[1]])
X_test = np.reshape(X_test,[X_test.shape[0],X_test.shape[2],X_test.shape[1]])

In [ ]:
display(X_train.shape)
_, features, seq_len = X_train.shape

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
labels = np.unique(y_train)
transform = {}
for i, l in enumerate(labels): transform[l] = i
y_train = np.vectorize(transform.get)(y_train)
y_val = np.vectorize(transform.get)(y_val)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

Checking shapes

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

UCR:(torch.Size([160, 10, 400]), (160,), torch.Size([74, 10, 400]), (74,))

ECG:(torch.Size([17111, 12, 1000]),(17111, 5),torch.Size([2156, 12, 1000]),(2156, 5))

## Instantiating the model

In [ ]:
n_kernels=10_000
kss=[7, 9, 11]
model = ROCKET(features, seq_len, n_kernels=n_kernels, kss=kss).to(device)

## Extracting features

In [ ]:
X_train_tfm = model(X_train).cpu().numpy()
X_val_tfm = model(X_val).cpu().numpy()

Saving features

In [ ]:
with open(path_to_data + 'features_X_train.pkl','wb') as handle:
    pickle.dump(X_train_tfm,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open(path_to_data + 'features_X_val.pkl','wb') as handle:
    pickle.dump(X_val_tfm,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open(path_to_data + 'y_train.pkl','wb') as handle:
    pickle.dump(y_train,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open(path_to_data + 'y_val.pkl','wb') as handle:
    pickle.dump(y_val,handle,protocol=pickle.HIGHEST_PROTOCOL)

Loading features

In [ ]:
path_to_data + 'y_val.pkl'

In [ ]:
with open(path_to_data + 'features_X_train.pkl','rb') as handle:
    X_train_tfm = pickle.load(handle)

with open(path_to_data + 'features_X_val.pkl','rb') as handle:
    X_val_tfm = pickle.load(handle)

In [ ]:
with open(path_to_data + 'y_train.pkl','rb') as handle:
    y_train = pickle.load(handle)

with open(path_to_data + 'y_val.pkl','rb') as handle:
    y_val = pickle.load(handle)

In [ ]:
X_train_tfm = X_train_tfm.reshape((X_train_tfm.shape[0],X_train_tfm.shape[1],1))
X_val_tfm = X_val_tfm.reshape((X_val_tfm.shape[0],X_val_tfm.shape[1],1))
y_train = y_train.reshape((y_train.shape[0], 1, y_train.shape[1]))
y_val = y_val.reshape((y_val.shape[0], 1, y_val.shape[1]))

In [ ]:
X_train_tfm.shape, y_train.shape, y_val.shape

## Actially running a classifier on the features

### MLP

### Pytorch MLP

In [ ]:
X_train_list = [l.astype(np.float32) for l in X_train_tfm]
X_val_list = [l.astype(np.float32) for l in X_val_tfm]
y_train_list = [l.astype(np.float32) for l in y_train]
y_val_list = [l.astype(np.float32) for l in y_val]

In [ ]:
X_train_tfm.shape, X_val_tfm.shape, y_train.shape, y_val.shape

In [ ]:
len(X_train_list),len(y_train_list),len(X_val_list)

In [ ]:
from fastai import *
from fastai.basic_data import *
from fastai.basic_train import *
from fastai.train import *
from fastai.metrics import *
from fastai.torch_core import *
metrics = []
wd=1e-2
input_size=X_train_tfm.shape[1]
#batchsize
bs=128

chunkify_train=False
chunkify_valid=True
chunk_length_valid=input_size
min_chunk_length=input_size#chunk_length

stride_length_train=input_size#chunk_length_train//8
stride_length_valid=input_size//2#chunk_length_valid


df_train = pd.DataFrame({"data":range(len(X_train_list)),"label":y_train_list})
df_valid = pd.DataFrame({"data":range(len(X_val_list)),"label":y_val_list})

tfms_ptb_xl = [ToTensor()]

ds_train=TimeseriesDatasetCrops(df_train,input_size,num_classes=5,chunk_length= 0,min_chunk_length=min_chunk_length,stride=stride_length_train,transforms=tfms_ptb_xl,annotation=False,col_lbl ="label",npy_data=X_train_list)
ds_valid=TimeseriesDatasetCrops(df_valid,input_size,num_classes=5,chunk_length=chunk_length_valid,min_chunk_length=min_chunk_length,stride=stride_length_valid,transforms=tfms_ptb_xl,annotation=False,col_lbl ="label",npy_data=X_val_list)


In [ ]:
db = DataBunch.create(ds_train,ds_valid,bs=bs)

In [ ]:
# Multilayer perceptron
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(20000, 3000, bias=True) 
        self.lin2 = nn.Linear(3000, 3000, bias=True)
        self.lin3 = nn.Linear(3000, 1000, bias=True)
        self.lin4 = nn.Linear(1000, 5, bias=True)

    def forward(self, xb):
        xb = xb.float()
        xb = F.relu(self.lin1(xb))
        xb = F.relu(self.lin2(xb))
        xb = F.relu(self.lin3(xb))
        xb = self.lin4(xb)
        return xb#xb.view(xb.size()[0], 5)

In [ ]:
model = MultilayerPerceptron() #nn.Sequential(*[nn.Linear(20_000, dls.c,), Squeeze(1)])
loss = F.binary_cross_entropy_with_logits

#learn = Learner(db,model, loss_func=loss, metrics=metrics,wd=wd,path='/content/models')
learn = Learner(db, model, metrics=accuracy, loss_func=loss)

learn.save('stage-0')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
X_val_tfm.shape

In [ ]:
X_val_tensor = torch.tensor(X_val_tfm[4], dtype=torch.float32, device=device)
model(X_val_tensor)

In [ ]:
learn.load('stage-0')
learn.fit_one_cycle(10, max_lr=3e-3, wd=1e2)
learn.recorder.plot_loss()

## Evaluating

In [ ]:
y_val.shape,pred.shape

In [ ]:
df_result = utils.evaluate_experiment(y_val,pred)
df_result